In [1]:
from main import get_miner
from log_parsing import from_id_to_template
import joblib
from data_integrate import *
from pattern_miner import *
from pattern_ranker import *

In [2]:
log_template_miner = get_miner()

In [3]:
from_id_to_template(20, log_template_miner)

'TraceID: <:TRACEID:> SpanID: <:SPANID:> Get currency data successful Service: currencyservice'

In [4]:
log_templates = []
for i in range(1, 1000):
    try:
        r = from_id_to_template(i, log_template_miner)
        if r != '':
            log_templates.append((i, r))
    except Exception as e:
        print(i, e)


[ERROR]2024-11-10 20:22:15,123 log_parsing.py:78: Cannot find id 213 's template
[ERROR]2024-11-10 20:22:15,123 log_parsing.py:78: Cannot find id 214 's template
[ERROR]2024-11-10 20:22:15,123 log_parsing.py:78: Cannot find id 215 's template
[ERROR]2024-11-10 20:22:15,123 log_parsing.py:78: Cannot find id 216 's template
[ERROR]2024-11-10 20:22:15,138 log_parsing.py:78: Cannot find id 217 's template
[ERROR]2024-11-10 20:22:15,138 log_parsing.py:78: Cannot find id 218 's template
[ERROR]2024-11-10 20:22:15,139 log_parsing.py:78: Cannot find id 219 's template
[ERROR]2024-11-10 20:22:15,140 log_parsing.py:78: Cannot find id 220 's template
[ERROR]2024-11-10 20:22:15,140 log_parsing.py:78: Cannot find id 221 's template
[ERROR]2024-11-10 20:22:15,140 log_parsing.py:78: Cannot find id 222 's template
[ERROR]2024-11-10 20:22:15,140 log_parsing.py:78: Cannot find id 223 's template
[ERROR]2024-11-10 20:22:15,140 log_parsing.py:78: Cannot find id 224 's template
[ERROR]2024-11-10 20:22:15,1

In [5]:
log_templates

[(1, 'frontend hipstershop.Frontend/Recv. start Service: frontend'),
 (2,
  'TraceID: <:TRACEID:> SpanID: <:SPANID:> Request complete Service: frontend'),
 (3,
  'TraceID: <:TRACEID:> SpanID: <:SPANID:> View user cart <:*:> Service: frontend'),
 (4,
  'TraceID: <:TRACEID:> SpanID: <:SPANID:> Request started Service: frontend'),
 (5, 'frontend hipstershop.Frontend/Recv. end Service: frontend'),
 (6,
  'frontend hipstershop.CurrencyService/GetSupportedCurrencies start Service: frontend'),
 (7,
  'frontend hipstershop.CurrencyService/GetSupportedCurrencies end Service: frontend'),
 (8, 'frontend hipstershop.CartService/GetCart start Service: frontend'),
 (9, 'frontend hipstershop.CartService/GetCart end Service: frontend'),
 (10,
  'frontend hipstershop.RecommendationService/ListRecommendations start Service: frontend'),
 (11,
  'frontend hipstershop.RecommendationService/ListRecommendations end Service: frontend'),
 (12,
  'frontend hipstershop.ProductCatalogService/GetProduct start Serv

error ids: 

In [6]:
'A'.lower()

'a'

In [7]:
err_ids = []
for t in log_templates:
    log = t[1]
    if 'err' in log.lower() or 'fail' in log.lower() or 'exception' in log.lower():
        print(t)
        err_ids.append(t[0])

(185, 'TraceID: <:TRACEID:>, SpanID: <:SPANID:> Request error Service: frontend')
(187, 'TraceID: <:TRACEID:> SpanID: <:SPANID:> Failed to complete the order Service: frontend')
(188, 'TraceID: <:TRACEID:> SpanID: <:SPANID:> Failed to connect payment service: connect error. Service: checkoutservice')
(189, 'TraceID: <:TRACEID:> SpanID: <:SPANID:> Could not charge the card: charge card error. Service: checkoutservice')
(201, 'TraceID: <:TRACEID:> SpanID: <:SPANID:> Failed to retrieve ads Service: frontend')
(204, 'TraceID: <:TRACEID:> SpanID: <:SPANID:> Failed to get product recommendations Service: frontend')
(205, 'TraceID: <:TRACEID:> SpanID: <:SPANID:> Failed to convert product Service: frontend')
(207, 'TraceID: <:TRACEID:> SpanID: <:SPANID:> Failed to prepare order: failed to get product #"<:PRODUCTID:>" Service: checkoutservice')
(208, 'TraceID: <:TRACEID:> SpanID: <:SPANID:> Failed to get product #"<:PRODUCTID:>" Service: checkoutservice')
(209, 'TraceID: <:TRACEID:> SpanID: <:S

In [8]:
err_ids

[185, 187, 188, 189, 201, 204, 205, 207, 208, 209]

In [9]:
res_dict = joblib.load('output/fault_free_eventgraphs.joblib')

In [10]:
res_dict2 = joblib.load('output/fault_suffering_eventgraphs.joblib')

In [11]:
fault_free_pattern = get_pattern_support(res_dict['2022-08-22 03:51'])

In [12]:
res_dict2['2022-08-22 05:02']['inject_pod']

'checkoutservice-578fcf4766-9csqn'

In [13]:
fault_suffering_event_graphs = res_dict2['2022-08-22 05:02']['event_graphs']
fault_suffering_pattern = get_pattern_support(fault_suffering_event_graphs)

In [16]:
r = pattern_ranker_custom(fault_free_pattern, fault_suffering_event_graphs, fault_suffering_pattern, log_template_miner)

In [18]:
r[1]

[{'events': '2_185',
  'score': 24,
  'deepth': 1,
  'pod': 'frontend-579b9bff58-t2dbm'},
 {'events': '153_188',
  'score': 24,
  'deepth': 1,
  'pod': 'frontend-579b9bff58-t2dbm'}]